In [1]:
from datetime import datetime

import findspark
findspark.init()

import pyspark

conf = pyspark.SparkConf()
print(conf)
conf.setMaster("local[*]")\
    .setAppName("My T40I10D100K")\
    .set("spark.driver.cores", 8)\
    .set("spark.driver.memory", "32g")\
    .set("spark.executor.cores", 8)\
    .set("spark.executor.memory" ,"32g")\
    .set("spark.executor.extraJavaOptions" ,"-XX:+UseG1GC")\
    .set("spark.memory.storageFraction", 0.8)
print(conf)

sc = pyspark.SparkContext(conf=conf)
sc.setLogLevel("INFO")
print(sc)
        
from pyspark.ml.fpm import FPGrowth
print(FPGrowth)

<SparkContext master=local[*] appName=My T40I10D100K>
<class 'pyspark.ml.fpm.FPGrowth'>


In [2]:
import csv
row_index = 0
avg_col_len = 0
max_col_len = 0
data= []
with open('data/T40I10D100K.dat', newline='') as csvfile:
  reader = csv.reader(csvfile, delimiter=' ', quotechar='|')
  for row in reader:
    array = [int(x) for x in row if x != '']
    avg_col_len = avg_col_len + len(array)
    if len(array) > max_col_len: max_col_len = len(array)
    data.append((row_index, array))
    row_index = row_index + 1
# data

In [3]:
avg_col_len = avg_col_len / row_index
# print(row_index, avg_col_len, max_col_len)
# print(data)

numSlices = 75
data = sc.parallelize(data, numSlices=numSlices)

params = ["id", "items"]

from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
df = sqlContext.createDataFrame(data, params)
df.cache()

row_index, avg_col_len, max_col_len

(100000, 39.60507, 77)

In [4]:
minSupport = 0.0025
minConfidence = 0.1
numPartitions = 1

start = datetime.now()

fpGrowth = FPGrowth(itemsCol="items", 
                    minSupport=minSupport, 
                    minConfidence=minConfidence, 
                    numPartitions=numPartitions)  

model = fpGrowth.fit(df)

model

FPGrowth_43aea880afe6dcae123b

In [5]:
# -- Count frequent itemsets.

model.freqItemsets.count()

7617884

In [6]:
# -- Display frequent itemsets.

# model.freqItemsets.show(5)

In [7]:
# -- Count generated association rules.

model.associationRules.count()

63949482

In [8]:
# -- Display generated association rules.

# model.associationRules.show(5)

In [9]:
# -- Count: transform examines the input items against all the association rules and summarize the
# -- consequents as prediction

# model.transform(df).count()

In [10]:
# -- transform examines the input items against all the association rules and summarize the
# -- consequents as prediction

# model.transform(df).show(5)

In [11]:
# sc.stop()

In [12]:
end = datetime.now()

diff = end - start

days, hours, minutes = diff.days, diff.seconds // 3600, diff.seconds % 3600 / 60.0

days, hours, minutes

(0, 1, 21.55)